In [3]:
%pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from pathlib import Path

In [6]:
def create_folder(path):
    """Cria uma pasta se ela não existir."""
    Path(path).mkdir(parents=True, exist_ok=True)

def download_image(url, path):
    """Baixa a imagem da URL se ela não existir ainda."""
    if Path(path).exists():
        return False
    try:
        resp = requests.get(url, timeout=10)
        if resp.status_code == 200:
            with open(path, "wb") as f:
                f.write(resp.content)
            return True
    except Exception as e:
        print(f"Erro ao baixar {url}: {e}")
    return False

In [7]:
# URL da listagem de trainers
url = "https://play.pokemonshowdown.com/sprites/trainers/"

# Faz o request e parseia o HTML
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

# Pega todos os <a href="xxx.png">
trainer_files = [
    a["href"] for a in soup.find_all("a", href=True)
    if a["href"].endswith(".png") and not a["href"].startswith("..")
]

print(f"Encontrados {len(trainer_files)} sprites de treinadores.")
# Exemplo: ['aaron.png', 'acerola-masters.png', ...]

Encontrados 1420 sprites de treinadores.


In [13]:
# Célula 4 – Baixar sprites de treinadores com limite de downloads únicos
from pathlib import Path

base_url    = "https://play.pokemonshowdown.com/sprites/trainers/"
destiny_dir = Path("trainers")
create_folder(destiny_dir)

# Defina aqui quantas imagens novas você quer baixar (não conta as que já existem)
max_images = int(input("Quantas imagens únicas deseja baixar? "))
count = 0

print(f"Iniciando download (até {max_images} novas imagens)...")
for fname in tqdm(trainer_files):
    src = f"{base_url}{fname}"
    dst = destiny_dir / fname

    # pula se já existir
    if dst.exists():
        continue

    # faz o download e conta apenas se foi realmente baixada
    if download_image(src, dst):
        count += 1
        # verifica se bateu o limite
        if count >= max_images:
            print(f"⏹️ Limite de {max_images} imagens atingido.")
            break

print(f"\nDownload concluído! Novas imagens baixadas: {count}")

Iniciando download (até 100 novas imagens)...


  0%|          | 0/1420 [00:00<?, ?it/s]

⏹️ Limite de 100 imagens atingido.

Download concluído! Novas imagens baixadas: 100
